In [13]:
import numpy as np

import lincoln
from lincoln.numpy.layers import Dense
from lincoln.numpy.losses import SoftmaxCrossEntropy, MeanSquaredError, SoftmaxCrossEntropyComplex
from lincoln.numpy.optimizers import Optimizer, SGD, SGDMomentum
from lincoln.numpy.activations import Sigmoid, Tanh, Linear, ReLU
from lincoln.numpy.network import NeuralNetwork
from lincoln.numpy.train import Trainer
from lincoln.data import mnist
from lincoln.numpy.layers import Conv2D
from lincoln.np_utils import softmax

X_train, y_train, X_test, y_test = mnist.load()

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
X_train, X_test = X_train - np.mean(X_train), X_test - np.mean(X_train)
X_train, X_test = X_train / np.std(X_train), X_test / np.std(X_train)

In [16]:
X_train_conv, X_test_conv = X_train.reshape(-1, 1, 28, 28), X_test.reshape(-1, 1, 28, 28)

In [17]:
num_labels = len(y_train)
train_labels = np.zeros((num_labels, 10))
for i in range(num_labels):
    train_labels[i][y_train[i]] = 1

num_labels = len(y_test)
test_labels = np.zeros((num_labels, 10))
for i in range(num_labels):
    test_labels[i][y_test[i]] = 1

In [18]:
def calc_accuracy_model(model, test_set):
    return print(f'''The model validation accuracy is: 
    {np.equal(np.argmax(model.forward(test_set, inference=True), axis=1), y_test).sum() * 100.0 / test_set.shape[0]:.2f}%''')

# CNN From Scratch

Below I show a convolutional neural network using the convolution operation implemented using a batch matrix multiply. See the bottom of the [Code](Code.ipynb) notebook for details.

In [8]:
np.sqrt(70765 / 275)

16.041423650264733

In [22]:
model = NeuralNetwork(
    layers=[Conv2D(out_channels=16,
                   param_size=5,
                   dropout=0.8,
                   weight_init="glorot",
                   flatten=True,
                  activation=Tanh()),
            Dense(neurons=10, 
                  activation=Linear())],
            loss = SoftmaxCrossEntropy(), 
seed=20190402)

trainer = Trainer(model, SGDMomentum(lr = 0.1, momentum=0.9))
trainer.fit(X_train_conv, train_labels, X_test_conv, test_labels,
            epochs = 1,
            eval_every = 1,
            seed=20190402,
            batch_size=60,
            conv_testing=True);

batch 0 loss 31.19150189374252
batch 10 loss 14.150390490520444
batch 20 loss 8.507022910130539
batch 30 loss 9.816084605553888
batch 40 loss 2.70693890403282
batch 50 loss 5.039137194847854
batch 60 loss 3.8413085692381324
batch 70 loss 8.476435226857522
batch 80 loss 5.382950790290696
batch 90 loss 2.3409849435175505
batch 100 loss 3.7577967015822247
Validation accuracy after 100 batches is 87.34%
batch 110 loss 8.835042029580162
batch 120 loss 6.248299860767069
batch 130 loss 6.094304954272118
batch 140 loss 2.2963053850591852
batch 150 loss 9.372588854516968
batch 160 loss 4.553819489115189
batch 170 loss 2.7017474375531543
batch 180 loss 4.144653939799545
batch 190 loss 5.302946416009569
batch 200 loss 6.2159890006486505
Validation accuracy after 200 batches is 85.91%
batch 210 loss 6.197134213166534
batch 220 loss 4.83542921877168
batch 230 loss 5.437383165865875
batch 240 loss 3.5524872075073803
batch 250 loss 3.4589611479364204
batch 260 loss 6.728340009816592
batch 270 loss 8.

In [60]:
calc_accuracy_model(model, X_test)

The model validation accuracy is: 
    89.37%


In [11]:
model = NeuralNetwork(
    layers=[Conv2D(out_channels=32,
                   param_size=5,
                   dropout=0.8,
                   weight_init="glorot",
                   flatten=True,
                  activation=Tanh()),
            Dense(neurons=10, 
                  activation=Linear())],
            loss = SoftmaxCrossEntropy(), 
seed=20190402)

trainer = Trainer(model, SGDMomentum(lr = 0.1, momentum=0.9))
trainer.fit(X_train, train_labels, X_test, test_labels,
            epochs = 1,
            eval_every = 1,
            seed=20190402,
            batch_size=60,
            conv_testing=True);

batch 0 loss 33.216544562324664
batch 10 loss 6.721844280652351
batch 20 loss 8.171938545760652
batch 30 loss 7.790772211152649
batch 40 loss 2.763450996757522
batch 50 loss 4.87810481499189
batch 60 loss 4.573422932170181
batch 70 loss 6.217209401215738
batch 80 loss 4.482600310761919
batch 90 loss 1.559477211993213
batch 100 loss 4.156000259467634
Validation accuracy after 100 batches is 86.02%
batch 110 loss 7.602353756623126


KeyboardInterrupt: 

### Comparing to our best fully connected one layer neural network

In [20]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Tanh(),
                  weight_init="glorot",
                  dropout=0.8),
            Dense(neurons=10, 
                  activation=Linear(),
                  weight_init="glorot")],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

trainer = Trainer(model, SGDMomentum(0.2, momentum=0.9, final_lr = 0.05, decay_type='exponential'))
trainer.fit(X_train, train_labels, X_test, test_labels,
       epochs = 1,
       eval_every = 1,
       seed=20190119,
           batch_size=60,
           early_stopping=True,
           conv_testing=True);

calc_accuracy_model(model, X_test)

batch 0 loss 3.1228996380817486
batch 10 loss 0.8932136825185367
batch 20 loss 0.660195193867239
batch 30 loss 0.5698901122343973
batch 40 loss 0.4018894515299986
batch 50 loss 0.44525706826989125
batch 60 loss 1.2854861072936496
batch 70 loss 0.46844966716006137
batch 80 loss 0.48728355225641284
batch 90 loss 0.8392205618217854
batch 100 loss 0.7123218138821693
Validation accuracy after 100 batches is 86.99%
batch 110 loss 0.8647573972048214
batch 120 loss 0.5344877210323652
batch 130 loss 1.0108547562185006
batch 140 loss 0.67596169501226
batch 150 loss 0.550133359711308
batch 160 loss 0.5832240401553612
batch 170 loss 0.5093058556547023
batch 180 loss 0.9334907163600867
batch 190 loss 0.7831829932837465
batch 200 loss 0.6795085067088273
Validation accuracy after 200 batches is 86.62%
batch 210 loss 0.11708777127774148
batch 220 loss 0.3458666311780603
batch 230 loss 0.696441179800066
batch 240 loss 0.6271036770305358
batch 250 loss 1.2383168801567237
batch 260 loss 0.983095136973540